# Transfer Learning with TensorFlow Part 1: feature Extraction

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problem

There are two mais benefits:

* Can leverage an existing neural network architecture proven to work on problems similar to our own.
* Can leverage a working neural netwrok architecture wich has already learned patterns on similar data to our own, then we can adapt those patterns to our own data.

In [24]:
# Célula de verificação APÓS instalar tensorflow-intel
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'

import tensorflow as tf
import numpy as np

print("🎯 TENSORFLOW-INTEL INSTALADO!")
print("="*50)

# Verificação específica do tensorflow-intel
print(f"Versão: {tf.__version__}")
print(f"Local: {tf.__file__}")

# Teste CONFIRMADO de oneDNN
print("\n✅ Teste de Matriz 2000x2000:")
size = 2000
a = tf.random.normal((size, size), dtype=tf.float32)
b = tf.random.normal((size, size), dtype=tf.float32)

import time
start = time.perf_counter()
c = tf.linalg.matmul(a, b)
_ = c.numpy()
elapsed = time.perf_counter() - start

gflops = (2 * size**3) / (elapsed * 1e9)
print(f"   • Tempo: {elapsed:.3f}s")
print(f"   • Performance: {gflops:.1f} GFLOPs")

# Classificação
if gflops > 40:
    print(f"   • Status: 🚀 EXCELENTE (oneDNN ativo)")
elif gflops > 20:
    print(f"   • Status: 👍 BOM")
else:
    print(f"   • Status: ⚠️  oneDNN inativo")

print("\n💡 DICA: Com tensorflow-intel, você deve ver:")
print("   • 'mkl' ou 'oneDNN' nas build flags")
print("   • Performance 20-50% melhor que NumPy")
print("   • GFLOPs acima de 30 no Ryzen 9")


🎯 TENSORFLOW-INTEL INSTALADO!
Versão: 2.15.0
Local: c:\Project_1\ml_ai\Lib\site-packages\tensorflow\__init__.py

✅ Teste de Matriz 2000x2000:
   • Tempo: 0.020s
   • Performance: 782.1 GFLOPs
   • Status: 🚀 EXCELENTE (oneDNN ativo)

💡 DICA: Com tensorflow-intel, você deve ver:
   • 'mkl' ou 'oneDNN' nas build flags
   • Performance 20-50% melhor que NumPy
   • GFLOPs acima de 30 no Ryzen 9


In [23]:
# ============================================================================
# 🦾 GEEKOM A9 MAX - 96GB RAM - CONFIGURAÇÃO SEGURA 🚀
# Célula INICIAL segura (sem %%time que crasha kernel)
# ============================================================================

import os
import sys
import warnings

# ================= CONFIGURAÇÃO ANTI-CRASH =================
# DESATIVE VALIDAÇÕES QUE CAUSAM KERNEL DEATH
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'  # MAIS IMPORTANTE!
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'           # Só erros
warnings.filterwarnings('ignore')

# ================= CONFIGURAÇÃO SEGURA PARA 96GB =================
# REDUZA THREADS INICIALMENTE para evitar overcommit
os.environ.update({
    # Otimizações oneDNN/AMD (mas seguras)
    'TF_ENABLE_ONEDNN_OPTS': '1',
    
    # THREADS REDUZIDOS inicialmente (evita crash)
    'OMP_NUM_THREADS': '16',           # 16 threads (não 48!)
    'MKL_NUM_THREADS': '8',            # 8 threads MKL
    'NUMEXPR_NUM_THREADS': '8',
    
    # Otimizações seguras de memória
    'TF_FORCE_GPU_ALLOW_GROWTH': 'true',
    'PYTHONUNBUFFERED': '1',
    
    # Cache diretórios (criar depois)
    'TFHUB_CACHE_DIR': 'C:/tfhub_cache',
    'TRANSFORMERS_CACHE': 'C:/huggingface_cache',
})

print("=" * 80)
print("🦾 GEEKOM A9 MAX - CONFIGURAÇÃO SEGURA INICIADA")
print("=" * 80)

# ================= IMPORTS BÁSICOS PRIMEIRO =================
import psutil
import numpy as np

# Info RAM
ram_gb = psutil.virtual_memory().total / (1024**3)
print(f"💾 MEMÓRIA: {ram_gb:.0f}GB RAM DISPONÍVEL")
print(f"🖥️  CPU: AMD Ryzen 9 8945HS (24 cores)")
print(f"🔧 Configuração: 16 threads inicial (seguro)")

# ================= IMPORT TENSORFLOW SEGURO =================
try:
    import tensorflow as tf
    print(f"\n📦 TensorFlow-intel {tf.__version__} carregado com sucesso")
    
    # Verificação básica
    print(f"🎯 Threads configurados: {os.environ.get('OMP_NUM_THREADS')}")
    print(f"🔧 oneDNN: {'✅ ATIVADO' if os.environ.get('TF_ENABLE_ONEDNN_OPTS') == '1' else '❌'}")
    
except ImportError as e:
    print(f"\n❌ ERRO ao importar TensorFlow: {e}")
    print("💡 Execute: pip install tensorflow-intel==2.15.0")
    tf = None
except Exception as e:
    print(f"\n⚠️  Aviso TensorFlow: {e}")
    tf = None

# ================= TESTE LEVE (OPCIONAL) =================
if tf is not None:
    print(f"\n🧪 Teste leve de funcionamento...")
    try:
        # Teste MUITO leve para não crashar
        size = 100  # Pequeno!
        a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        b = tf.constant([[5.0, 6.0], [7.0, 8.0]])
        c = tf.matmul(a, b)
        print(f"✅ Operação básica OK: {c.numpy()}")
        
    except Exception as e:
        print(f"⚠️  Teste básico falhou: {e}")

# ================= ATUALIZAÇÃO PARA MODO 96GB (APÓS IMPORTS) =================
print(f"\n⚡ AGORA você pode usar toda a potência:")
print(f"   1. Aumentar threads gradualmente")
print(f"   2. Criar datasets gigantes")
print(f"   3. Usar batch_size grande")

# ================= FUNÇÃO PARA ATIVAR MODO BEAST (usar depois) =================
def ativar_modo_beast():
    """Ativa configuração máxima 96GB - USAR DEPOIS que kernel está estável"""
    print("\n" + "🔥" * 40)
    print("🔥 ATIVANDO MODO BEAST 96GB!")
    print("🔥" * 40)
    
    # Aumenta threads para máximo
    os.environ.update({
        'OMP_NUM_THREADS': '48',
        'MKL_NUM_THREADS': '48',
        'NUMEXPR_NUM_THREADS': '48',
        'OPENBLAS_NUM_THREADS': '48',
    })
    
    # Configurações avançadas
    os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2'
    os.environ['TF_GPU_MEMORY_LIMIT'] = str(80 * 1024**3)  # 80GB
    
    # Criar cache gigante
    cache_dirs = ['C:/tfhub_cache_96gb', 'C:/huggingface_cache_96gb']
    for dir_path in cache_dirs:
        os.makedirs(dir_path, exist_ok=True)
        os.environ['TFHUB_CACHE_DIR'] = dir_path
    
    print(f"🎯 Threads aumentados para: 48")
    print(f"💾 Cache 96GB configurado")
    print(f"🚀 PRONTO PARA DATASETS GIGANTES!")
    
    return True

# ================= FUNÇÃO PARA TESTE DE PERFORMANCE (usar depois) =================
def teste_performance_96gb(size=2000):
    """Teste de performance - USAR DEPOIS que kernel está estável"""
    if tf is None:
        print("❌ TensorFlow não carregado")
        return
    
    print(f"\n🧪 Teste performance matriz {size}x{size}...")
    try:
        import time
        a = tf.random.normal((size, size), dtype=tf.float32)
        b = tf.random.normal((size, size), dtype=tf.float32)
        
        start = time.perf_counter()
        c = tf.linalg.matmul(a, b)
        _ = c.numpy()
        elapsed = time.perf_counter() - start
        
        gflops = (2 * size**3) / (elapsed * 1e9)
        print(f"✅ {size}x{size}: {elapsed:.2f}s ({gflops:.1f} GFLOPs)")
        print(f"💾 Memória usada: ~{(size*size*4*2)/1024**2:.0f}MB")
        
    except Exception as e:
        print(f"⚠️  Teste falhou: {e}")

# ================= RECOMENDAÇÕES DE USO =================
print(f"\n" + "🎯" * 40)
print("🎯 COMO USAR ESTE AMBIENTE:")
print("🎯" * 40)
print(f"1. Esta célula já carregou TensorFlow com configuração SEGURA")
print(f"2. Para ATIVAR MODO 96GB COMPLETO, execute:")
print(f"   ativar_modo_beast()")
print(f"3. Para TESTAR PERFORMANCE, execute:")
print(f"   teste_performance_96gb(5000)  # 5Kx5K matriz")
print(f"4. Agora importe outras bibliotecas normalmente")
print(f"\n✅ AMBIENTE INICIALIZADO COM SUCESSO!")
print("=" * 80)

🦾 GEEKOM A9 MAX - CONFIGURAÇÃO SEGURA INICIADA
💾 MEMÓRIA: 92GB RAM DISPONÍVEL
🖥️  CPU: AMD Ryzen 9 8945HS (24 cores)
🔧 Configuração: 16 threads inicial (seguro)

📦 TensorFlow-intel 2.15.0 carregado com sucesso
🎯 Threads configurados: 16
🔧 oneDNN: ✅ ATIVADO

🧪 Teste leve de funcionamento...
✅ Operação básica OK: [[19. 22.]
 [43. 50.]]

⚡ AGORA você pode usar toda a potência:
   1. Aumentar threads gradualmente
   2. Criar datasets gigantes
   3. Usar batch_size grande

🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
🎯 COMO USAR ESTE AMBIENTE:
🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
1. Esta célula já carregou TensorFlow com configuração SEGURA
2. Para ATIVAR MODO 96GB COMPLETO, execute:
   ativar_modo_beast()
3. Para TESTAR PERFORMANCE, execute:
   teste_performance_96gb(5000)  # 5Kx5K matriz
4. Agora importe outras bibliotecas normalmente

✅ AMBIENTE INICIALIZADO COM SUCESSO!


In [3]:
# ============================================================================
# 📦 IMPORTS COMPLETOS - AGORA SEGURO
# ============================================================================

print("📦 Carregando bibliotecas completas...")

# Data Science básico
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print(f"✅ Pandas {pd.__version__}")

# TensorFlow Hub (se necessário)
try:
    import tensorflow_hub as hub
    print(f"✅ TensorFlow Hub {hub.__version__}")
except:
    print("⚠️  TensorFlow Hub não disponível")

# Scikit-learn
import sklearn
print(f"✅ Scikit-learn {sklearn.__version__}")

# Configurações visuais
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("🎯 Todas bibliotecas carregadas!")
print("💡 Dica: Agora você pode usar ativar_modo_beast()")

📦 Carregando bibliotecas completas...
✅ Pandas 2.2.1



✅ TensorFlow Hub 0.15.0
✅ Scikit-learn 1.4.2
🎯 Todas bibliotecas carregadas!
💡 Dica: Agora você pode usar ativar_modo_beast()


In [25]:
# ============================================================================
# 🔥 ATIVAÇÃO DO MODO 96GB - APÓS TUDO ESTÁVEL
# ============================================================================

print("⚡ Ativando potência total 96GB...")

# 1. Primeiro aumente threads gradualmente
os.environ['OMP_NUM_THREADS'] = '32'
os.environ['MKL_NUM_THREADS'] = '32'
print(f"✅ Threads aumentados para 32")

# 2. Importe bibliotecas pesadas
import tensorflow as tf
from tensorflow.keras import layers, models

# 3. Ative modo beast completo
ativar_modo_beast()

# 4. Teste performance
teste_performance_96gb(3000)  # Comece com 3Kx3K

print("\n🚀 MODO 96GB ATIVADO! PRONTO PARA:")
print("   • Batch sizes até 4096")
print("   • Datasets de 50GB+ em RAM")
print("   • Múltiplos modelos simultâneos")

⚡ Ativando potência total 96GB...
✅ Threads aumentados para 32

🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥
🔥 ATIVANDO MODO BEAST 96GB!
🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥
🎯 Threads aumentados para: 48
💾 Cache 96GB configurado
🚀 PRONTO PARA DATASETS GIGANTES!

🧪 Teste performance matriz 3000x3000...
✅ 3000x3000: 0.05s (1120.0 GFLOPs)
💾 Memória usada: ~69MB

🚀 MODO 96GB ATIVADO! PRONTO PARA:
   • Batch sizes até 4096
   • Datasets de 50GB+ em RAM
   • Múltiplos modelos simultâneos


In [3]:
# ============================================================================
# 🛠️ FUNÇÕES ESPECIAIS PARA 96GB RAM
# ============================================================================

import tensorflow as tf
import numpy as np
import psutil

def create_huge_dataset(dataset_size_gb=20):
    """Cria dataset gigante em RAM (só com 96GB!)"""
    elements = int((dataset_size_gb * 1024**3) / 4 / 784)  # Para MNIST-like
    x = tf.random.normal([elements, 28, 28, 1], dtype=tf.float32)
    y = tf.random.uniform([elements], maxval=10, dtype=tf.int32)
    print(f"✅ Dataset de {dataset_size_gb}GB criado: {x.shape}")
    return tf.data.Dataset.from_tensor_slices((x, y)).batch(1024)

def memory_status():
    """Status detalhado da memória"""
    mem = psutil.virtual_memory()
    print(f"💾 STATUS MEMÓRIA:")
    print(f"   • Total: {mem.total / 1024**3:.0f}GB")
    print(f"   • Disponível: {mem.available / 1024**3:.0f}GB")
    print(f"   • Usado: {mem.used / 1024**3:.0f}GB")
    print(f"   • Percentual: {mem.percent}%")

def train_large_batch(model, dataset, batch_size=2048):
    """Treina com batch_size gigante"""
    print(f"🚀 Treinando com batch_size={batch_size} (96GB power!)")
    # Sua lógica de treino aqui
    pass

# Verificação
memory_status()

💾 STATUS MEMÓRIA:
   • Total: 92GB
   • Disponível: 73GB
   • Usado: 19GB
   • Percentual: 20.5%


In [2]:
import tensorflow as tf
print(tf.__version__)

2.15.0


# Download and becoming one with the data

In [3]:
# # Get the data (10% of 10 food class from food101 dataset)
# # https://www.kaggle.com/datasets/dansbecker/food-101

# #import zipfile
# #import urllib.request

# #Baixar o arquivo ZIP
# url = "https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip"
# zip_path = "food_classes_10_percent.zip"
# urllib.request.urlretrieve(url, zip_path)

# # # Descompactar o arquivo ZIP
# with zipfile.ZipFile(zip_path, "r") as zip_ref:
#     zip_ref.extractall()

In [5]:
# How many images in each folder?
import os

#WEalk through 10 percent data directory and list number of file
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent\test'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent

## Create data loaders (preparing the data)

We'll use the `ImageDataGenerator` class to load in our images in batches

In [34]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1./255.)
test_datagen = ImageDataGenerator(rescale=1./255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(directory=train_dir,
                                                           target_size=IMAGE_SIZE,
                                                           batch_size=BATCH_SIZE,
                                                           class_mode="categorical")
print("Testing images:")
test_data = test_datagen.flow_from_directory(directory=test_dir,
                                                            target_size=IMAGE_SIZE,
                                                            batch_size=BATCH_SIZE,
                                                            class_mode="categorical")


Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


## Setting up callbacks (things to run whilst our model trains)

Callbacks are extra functionality you can add to your models to be performed during or after training. Some of the most popular callbacks:

* Tracking experiments with Tensorboard callback
* Model checkpoint with the ModelCheckpoint callback
* Stopping a model from training (before it trains too long and overfits) with the EarlyStopping callback

In [35]:
# Create TensorBoard callback (functionize because we need to create a new one for each model)
import datetime # to help create unique log directory names
def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # create log directory
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir) # create TensorBoard callback
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback

> Note: You can customize the directory where your TensorBoared logs (model training metrics) get saved to whatever you like.
The `log_dir` parameter we've created above is only one option.)

## Creating models using TensorFlow Hub

In the past we've used TensorFlow to create our own models layers by layer from scratch.
Now we're going to do a similar process, except the majority of our model's layers are going to come from TensorFlow Hub.
We can access pretrained models on: https://tfhub.dev/

Browsing the tensdorflow Hub page and sorting for image classification, we found the following feature vector model link: https://www.kaggle.com/models/google/efficientnet-v2


In [36]:
# Classificação ImageNet
resnet_v2_url = "https://tfhub.dev/tensorflow/resnet_50/classification/1"

# Ou para recursos (feature vectors):
resnet_feature_url = "https://tfhub.dev/tensorflow/resnet_50/feature_vector/1"

In [39]:
# let's make a create_model() function to reuse for each model

from os import name


def create_model(model_url, num_classes=10):
    """
    Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
    Args:
      model_url (str): A TensorFlow Hub feature extraction URL.
      num_classes (int): Number of output neurons/classes in the output layer, should be equal to Number of target classes,
        default 10.

    Returns:
      An uncompiled Keras Sequential model with model_url as feature extractor layer and Dense output layer  with num_classes output neurons. 
    """

    # Download the pretrained model and save it as a Keras layer
    feature_extractor_layer = hub.KerasLayer(model_url,
                                             name=f"feature_extractor_layer", 
                                             input_shape=IMAGE_SIZE+(3,), # add 3 color channels to the image size
                                             trainable=False) # freeze the already learned patterns

    # Create our own model
    model = tf.keras.Sequential([
            feature_extractor_layer,
            layers.Dense(num_classes, activation="softmax", dtype='float32', name="output_layer") # ensure output is float32
        ])

    return model

### Creating and testing ResNet TensorFlow Hub Feature Estraction model

In [40]:
# Create Resnet model
resnet_model = create_model(resnet_feature_url,
                             num_classes=train_data_10_percent.num_classes)

In [41]:
# Compile our resnet model
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [43]:
# Let's fit our resNet model to the data (10 percent of the food101 dataset)
resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                         experiment_name="resnet50V2_feature_extraction")])

Saving TensorBoard log files to: tensorflow_hub/resnet50V2_feature_extraction/20260117-173350
Epoch 1/5
24/24 [==============================] - 42s 2s/step - loss: 0.7634 - accuracy: 0.7960 - val_loss: 0.7248 - val_accuracy: 0.7664
Epoch 2/5
24/24 [==============================] - 42s 2s/step - loss: 0.5417 - accuracy: 0.8533 - val_loss: 0.6255 - val_accuracy: 0.8056
Epoch 3/5
24/24 [==============================] - 43s 2s/step - loss: 0.3993 - accuracy: 0.9027 - val_loss: 0.5828 - val_accuracy: 0.8204
Epoch 4/5
24/24 [==============================] - 43s 2s/step - loss: 0.3180 - accuracy: 0.9240 - val_loss: 0.5663 - val_accuracy: 0.8208
Epoch 5/5
24/24 [==============================] - 44s 2s/step - loss: 0.2629 - accuracy: 0.9547 - val_loss: 0.5529 - val_accuracy: 0.8204


In [16]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extractor_layer (K  (None, 2048)              23561152  
 erasLayer)                                                      
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23581642 (89.96 MB)
Trainable params: 20490 (80.04 KB)
Non-trainable params: 23561152 (89.88 MB)
_________________________________________________________________


That is Increadible!!!. Our transfer learning feature extractor model out performed ALL of the previous models we built by hand... (substantially) and quicker training time AND with only 10% of the training examples.

In [ ]:
# let's create a function to plot loss curves
#Tidbit: you could put this in a separate helper.py file and import it wherever you need it
import matplotlib.pyplot as plt
def plot_loss_curves(history):
    """
    Returns separate loss curves for training and validation metrics.
    Args:
      history: TensorFlow model History object (returned from model.fit()).
    Returns:
      Plots of training/validation loss and accuracy curves.
    """
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(len(history.history['loss']))

    # Plot loss
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='Training Accuracy')
    plt.plot(epochs, val_accuracy, label='Validation Accuracy')
    plt.title('Accuracy Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()